In [1]:
import json
import snowflake.connector

with open('config.json', 'r') as f:
    config = json.load(f)

conn = snowflake.connector.connect(**config)

In [24]:
sql = '''-- STG_MLS columns
SELECT
    m.CC_PROPERTY_ID,
    m.CC_RECORD_ID,
    m.CC_LIST_ID,
    m.CITY,
    m.STATE,
    m.ZIP,
    m.PRICE,
    m.ASSESSED_YEAR,
    m.ASSESSED_TOTAL_VALUE,
    m.TAX_AMOUNT,
    m.TAX_AMOUNT_FREQUENCY,
    m.YEAR_BUILT,
    m.EFFECTIVE_YEAR_BUILT,

    -- EXPANDED_IMAGE_DATA columns
    e.PROPERTY_ID

FROM SCRATCH.BASIS.STG_MLS m
LEFT JOIN SCRATCH.PUBLIC.EXPANDED_IMAGE_DATA e
    ON m.CC_LIST_ID = e.CC_LIST_ID limit 2000;'''

In [30]:
sql1 = '''SELECT
    m.PRICE,
    m.ASSESSED_TOTAL_VALUE
FROM SCRATCH.BASIS.STG_MLS m
LEFT JOIN SCRATCH.PUBLIC.EXPANDED_IMAGE_DATA e
    ON m.CC_LIST_ID = e.CC_LIST_ID
WHERE
    m.PRICE IS NOT NULL
    AND m.ASSESSED_TOTAL_VALUE IS NOT NULL
    AND m.ASSESSED_YEAR IN (2025, 2026);'''

In [32]:
cursor = conn.cursor()
cursor.execute(sql1)
df = cursor.fetch_pandas_all()

In [34]:
import plotly.express as px
import pandas as pd

# Scatter plot
fig = px.scatter(
    df,
    x='ASSESSED_TOTAL_VALUE',
    y='PRICE',
    title='Price vs Assessed Total Value',
    labels={'ASSESSED_TOTAL_VALUE': 'Assessed Total Value', 'PRICE': 'Price'},
    opacity=0.5
)

# Add y=x line
max_val = max(df['ASSESSED_TOTAL_VALUE'].max(), df['PRICE'].max())
fig.add_scatter(x=[0, max_val], y=[0, max_val], mode='lines', name='y=x', line=dict(color='red', dash='dash'))

fig.show()

In [19]:
df['price','assessed_total_value']

KeyError: ('price', 'assessed_total_value')

In [9]:
df.shape

(3, 90)

In [12]:
df[['CC_PROPERTY_ID', 'CC_RECORD_ID', 'CC_LIST_ID', 'CITY', 'STATE', 'ZIP','A_DATE','ASSESSED_TOTAL_VALUE','ASSESSED_YEAR','EFFECTIVE_YEAR_BUILT']]

,CC_PROPERTY_ID,CC_RECORD_ID,CC_LIST_ID,CITY,STATE,ZIP,A_DATE,ASSESSED_TOTAL_VALUE,ASSESSED_YEAR,EFFECTIVE_YEAR_BUILT
0,144428807,1489272110,46186100,woodbury,mn,55125,1999-10-08,NaN,1999,NaN
1,144420719,2412653768,46265133,woodbury,mn,55125,2004-10-15,101.0,2004,NaN
2,144287366,3517588604,160386631,woodbury,mn,55125,2011-03-24,281.0,2010,NaN


In [11]:
for i in df.columns:
    print(i)

CC_PROPERTY_ID
CC_RECORD_ID
CC_LIST_ID
CITY
STATE
ZIP
FIPS_COUNTY_CD
MOST_RECENT_SALE
TRENDED_PRIOR_SALE
MARKET_PPSF
STATUS
A_DATE
PRICE
SALE_TYPE
CONCESSIONS_INDICATOR
CONCESSIONS_AMOUNT
STANDARDIZED_PROPERTY_TYPE
MLS_PROPERTY_TYPE
MLS_PROPERTY_SUB_TYPE
PROPERTY_OWNERSHIP_TYPE
MLS_DAYS_ON_MARKET
MLS_CUMULATIVE_DAYS_ON_MARKET
ASSESSED_YEAR
ASSESSED_TOTAL_VALUE
TAX_AMOUNT
TAX_AMOUNT_FREQUENCY
YEAR_BUILT
EFFECTIVE_YEAR_BUILT
LOT_SQFT
LOT_SIZE_SOURCE
LOT_DIMENSIONS
LOT_FEATURE_LIST
GLA_SQFT
GLA_SQFT_SOURCE
FLOORS_IN_PROPERTY
FLOORS_IN_BUILDING
NUMBER_OF_UNITS
ROOMS
BEDROOMS
FULL_BATHS
HALF_BATHS
BASEMENT_TYPE
PROPERTY_CONDITION
DISCLOSURES_DESCRIPTION
PROPERTY_QUALITY
GARAGE_SPACES
GARAGE_TYPE
PARKING_FEATURES
POOL_SPA_TYPES
CURRENT_OCCUPANCY_TYPE
VIEW_DESCRIPTION
HEATING
COOLING
FIREPLACE_TYPE
FIREPLACE_COUNT
FOUNDATION_TYPE
ROOF_TYPE
HOME_STYLE
PORCH_OR_PATIO
UTILITIES_LIST
ELECTRICITY_INCLUDED
ELECTRICITY_DESCRIPTION
WATER_SOURCE
SEWER_DESCRIPTION
GAS_DESCRIPTION
OTHER_EQUIPMENT_INCLUD